In [3]:
import requests
import json
import math
import time
import os
import uuid
import platform
import logging
import subprocess
from subprocess import Popen, PIPE
import pandas as pd
from dateutil.relativedelta import relativedelta
import datetime
from datetime import timedelta
from dateutil import tz
import dateutil.parser as dp
import pandas.io.gbq as gbq
from google.cloud import bigquery
from google.cloud import storage

In [4]:
app_path = os.getcwd()
gspath = 'gs://sarasmaster'
os.chdir(os.getcwd())
filesep = '\\' if platform.system() == 'Windows' else '/'
filepath = app_path + filesep + 'kopari' + filesep + 'nordstrom'
gssep = '/'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "creds" + filesep + "sarasmaster-524142bf5547.json"
gcspath = 'C:\\Users\\kabhi\\AppData\\Local\\Google\\Cloud SDK\\google-cloud-sdk\\bin'
os.environ["PATH"] += os.pathsep + gcspath
batfile = app_path + filesep + 'movetogcs.bat' if platform.system() == 'Windows' else app_path + filesep + 'movetogcs.sh'
delimitertype = 'CSV'
loadtype = 'WRITE_APPEND'
skipheader = 1

In [1]:
def processdataframe(dfdata):
    dfnew = pd.DataFrame(columns=('business_unit','group','region','store_id','store',
    'vpn','last_receipt_date','sales','p_sales_to_total_supplier','sales_units',
    'receipts_retail','receipts_units','p_sell_thru_units','eoh','eoh_to_ttl_supplier',
    'eoh_units','p_reg_promo_eoh_units','oo_retail','oo_units'))

    businessunit=''
    group=''
    region=''
    store=''
    store_id = ''
    for idx,idx_row in dfdata.iterrows():
        row = dict()
        if idx_row[1] == 'Total Business Unit:':
            businessunit = idx_row[5]
            group = ''
            region = ''
            store_id = ''
            store = ''
        if idx_row[1] == 'Total Group:':
            group = idx_row[4]
            region = ''
            store_id = ''
            store = ''
        if idx_row[1] == 'Total Region:':
            region = idx_row[4]
            store_id = ''
            store = ''
        if idx_row[1] == 'Total Store:':
            store = idx_row[5]
            store_id = idx_row[3]
    
        row['business_unit'] = businessunit
        row['group'] = group
        row['region'] = region
        row['store'] = store
        row['store_id'] = store_id
        row['vpn'] = idx_row[2]
        row['last_receipt_date'] = idx_row[7]
        row['sales'] = idx_row[8]
        row['p_sales_to_total_supplier'] = idx_row[10]
        row['sales_units'] = idx_row[11]
        row['receipts_retail'] = idx_row[12]
        row['receipts_units'] = idx_row[13]
        row['p_sell_thru_units'] = idx_row[14]
        row['eoh'] = idx_row[16]
        row['eoh_to_ttl_supplier'] = idx_row[17]
        row['eoh_units'] = idx_row[19]
        row['p_reg_promo_eoh_units'] = idx_row[20]
        row['oo_retail'] = idx_row[22]
        row['oo_units'] = idx_row[24]
        row_s = pd.Series(row)    
        dfnew = dfnew.append(row_s, ignore_index=True)
    return dfnew

In [4]:
dfheaders.head()

NameError: name 'dfheaders' is not defined

In [18]:
for filename in os.listdir(filepath):
    if not filename.endswith('.xls'): continue
    fullname = os.path.join(filepath, filename)
    print(fullname)
    df = pd.read_excel(fullname, sheet_name=0,header=None)
    dfdata = df[9:]
    dfheaders = df[:9]
    dfdata = dfdata.reset_index(drop=True)
    dfdata = processdataframe(dfdata)
    start_date = dfheaders.iloc[0][5]
    end_date = dfheaders.iloc[0][8]
    no_week = start_date.isocalendar()[1]
    currency_cd = 'USD'
    quarter = pd.Timestamp(start_date).quarter
    year = pd.Timestamp(start_date).year
    load_id = uuid.uuid4()
    dfdata['load_id'] = load_id
    dfdata['start_date'] = start_date
    dfdata['end_date'] = end_date
    dfdata['year'] = year
    dfdata['qtr'] = quarter
    dfdata['week'] = no_week
    dfdata['currency_code'] = currency_cd
    dfdata = dfdata[(dfdata.store.notnull()) & (dfdata.vpn.notnull()) & (dfdata.store != '') & (dfdata.vpn != '')]
    dfdata.to_csv(fullname + '.csv', index=False)

C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom Sales Data week end 2_10_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom Sales Summary weekend 2_24_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom Sales Week End 4_7_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom Sales Week Ending 3_10_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom Sales Week Ending 3_17_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom Summary Sales 1_28 to 2_3_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom Week Ending 3_31_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom week ending 3_3_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom Week ending 4_22_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\Nordstrom1_13_18WEEKLYDATA.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\NordstromDataWeekEnd1_27_18.xls
C:\Users\kabhi\Desktop\smd\kopari\nordstrom\NordstromWeekending1_20_18 (1).xls
C:\Users\kabhi\Desktop\smd\k

In [13]:
df = pd.read_excel('C:\\Users\\kabhi\\Desktop\\smd\\kopari\\nordstrom\\Nordstrom Sales Data week end 2_10_18.xls', sheet_name=0,header=None)
dfdata = df[9:]
dfheaders = df[:9]
dfdata = dfdata.reset_index(drop=True)
dfdata = processdataframe(dfdata)

In [26]:
dfdata.to_csv('test2.csv')

In [14]:
dfnew.head()

NameError: name 'dfnew' is not defined

In [82]:
dfdata.head()

,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,24,group,bus_unit
0,NaN,Total Business Unit:,NaN,NaN,NaN,FULL LINE,NaN,NaN,704.14,NaN,...,1,393.0,NaN,1,11238.0,NaN,342.0,NaN,,NaN
1,NaN,NaN,"KCM-0200,COCONUT MELT 7OZ",NaN,NaN,NaN,NaN,2/23/2018,321.86,NaN,...,0.491434,154.0,NaN,1,6156.0,NaN,162.0,NaN,,
2,NaN,NaN,"KCD-0057,COCO DEODORANT",NaN,NaN,NaN,NaN,2/23/2018,237.16,NaN,...,0.0623111,53.0,NaN,1,0.0,NaN,0.0,NaN,,
3,NaN,NaN,"KOP-0014,COCO COLLECTION OIL PULLING",NaN,NaN,NaN,NaN,2/23/2018,171.92,NaN,...,0.300974,128.0,NaN,1,3948.0,NaN,141.0,NaN,,
4,NaN,NaN,"KBO-0100,COCONUT BODY OIL",NaN,NaN,NaN,NaN,4/26/2002,0,NaN,...,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,,


In [18]:
dfna.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,NaN,Total Business Unit:,NaN,NaN,NaN,FULL LINE,NaN,NaN,704.14,NaN,...,11908.0,NaN,1,393.0,NaN,1,11238.0,NaN,342.0,NaN
20,NaN,Total Group:,NaN,NaN,NORTHWEST FLS,NaN,NaN,NaN,14,NaN,...,1262.0,NaN,1,42.0,NaN,1,1392.0,NaN,39.0,NaN
32,NaN,Total Region:,NaN,NaN,WASHINGTON/ALASKA FU,NaN,NaN,NaN,14,NaN,...,740.0,NaN,1,25.0,NaN,1,1080.0,NaN,30.0,NaN
42,NaN,Total Store:,NaN,1.0,NaN,DT SEATTLE,NaN,NaN,0,NaN,...,262.0,NaN,1,9.0,NaN,1,768.0,NaN,21.0,NaN
47,NaN,Total Store:,NaN,4.0,NaN,BELLEVUE,NaN,NaN,14,NaN,...,358.0,NaN,1,11.0,NaN,1,312.0,NaN,9.0,NaN


In [11]:
dfdata.head()

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,NaN,Total Business Unit:,NaN,NaN,NaN,FULL LINE,NaN,NaN,704.14,NaN,...,11908.0,NaN,1,393.0,NaN,1,11238.0,NaN,342.0,NaN
1,NaN,NaN,"KCM-0200,COCONUT MELT 7OZ",NaN,NaN,NaN,NaN,2/23/2018,321.86,NaN,...,5852.0,NaN,0.491434,154.0,NaN,1,6156.0,NaN,162.0,NaN
2,NaN,NaN,"KCD-0057,COCO DEODORANT",NaN,NaN,NaN,NaN,2/23/2018,237.16,NaN,...,742.0,NaN,0.0623111,53.0,NaN,1,0.0,NaN,0.0,NaN
3,NaN,NaN,"KOP-0014,COCO COLLECTION OIL PULLING",NaN,NaN,NaN,NaN,2/23/2018,171.92,NaN,...,3584.0,NaN,0.300974,128.0,NaN,1,3948.0,NaN,141.0,NaN
4,NaN,NaN,"KBO-0100,COCONUT BODY OIL",NaN,NaN,NaN,NaN,4/26/2002,0,NaN,...,0.0,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


In [163]:
dfdata.columns = ['business_unit','group','region','store_id','store',
'vpn','last_receipt_date','sales','p_sales_to_total_supplier','sales_units',
'receipts_retail','receipts_units','p_sell_thru_units','eoh','eoh_to_ttl_supplier',
'eoh_units','p_reg_promo_eoh_units','oo_retail','oo_units']
dfdata['sku_number'], dfdata['item_description'] = dfdata['vpn'].str.split(',', 1).str

#dfdata['sku_number'],dfdata['item_description']  = pd.DataFrame(dfdata['vpn'].str.split(',',1).tolist(),columns = ['sku_number','item_description'])
#newdf  = pd.DataFrame(dfdata['vpn'].str.split(',',1).tolist(),columns = ['sku_number','item_description'])

In [164]:
dfdata.head()

,business_unit,group,region,store_id,store,vpn,last_receipt_date,sales,p_sales_to_total_supplier,sales_units,...,receipts_units,p_sell_thru_units,eoh,eoh_to_ttl_supplier,eoh_units,p_reg_promo_eoh_units,oo_retail,oo_units,sku_number,item_description
0,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KCM-0200,COCONUT MELT 7OZ",4/9/2018,938.6,0.471407,28,...,54,0.411765,1520,0.475,40,1,NaN,NaN,KCM-0200,COCONUT MELT 7OZ
1,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KCD-0057,COCO DEODORANT",4/19/2018,548.38,0.275421,43,...,123,0.323308,1260,0.39375,90,1,84,6,KCD-0057,COCO DEODORANT
2,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KOP-0014,COCO COLLECTION OIL PULLING",3/21/2018,308.56,0.154973,13,...,6,0.764706,112,0.035,4,1,84,3,KOP-0014,COCO COLLECTION OIL PULLING
3,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KSS-0100,SLIP INTO SUMMER KIT",3/30/2018,177.52,0.0891585,7,...,19,0.388889,308,0.09625,11,1,NaN,NaN,KSS-0100,SLIP INTO SUMMER KIT
4,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KCM-0075,COCONUT MINI MELT 2.5OZ",NaN,18,0.00904041,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,KCM-0075,COCONUT MINI MELT 2.5OZ


In [174]:
start_date = dfheaders.iloc[0][1]
end_date = dfheaders.iloc[0][3]
no_week = dfheaders.iloc[0][5]
currency_cd = 'USD' if '$' in dfheaders.iloc[1][7] else 'TBD'
quarter = pd.Timestamp(start_date).quarter
year = pd.Timestamp(start_date).year
load_id = uuid.uuid4()
dfdata['load_id'] = load_id
dfdata['start_date'] = start_date
dfdata['end_date'] = end_date
dfdata['year'] = year
dfdata['qtr'] = quarter
dfdata['week'] = no_week
dfdata['currency_code'] = currency_cd
print(start_date)
print(end_date)

2018-01-07 00:00:00
2018-04-21 00:00:00


In [175]:
dfdata.head()

,business_unit,group,region,store_id,store,vpn,last_receipt_date,sales,p_sales_to_total_supplier,sales_units,...,oo_units,sku_number,item_description,load_id,year,qtr,week,currency_code,start_date,end_date
0,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KCM-0200,COCONUT MELT 7OZ",4/9/2018,938.6,0.471407,28,...,NaN,KCM-0200,COCONUT MELT 7OZ,aa00f60f-d5ff-49de-8bd6-88a0434a7aca,2018,1,15,USD,2018-01-07,2018-04-21
1,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KCD-0057,COCO DEODORANT",4/19/2018,548.38,0.275421,43,...,6,KCD-0057,COCO DEODORANT,aa00f60f-d5ff-49de-8bd6-88a0434a7aca,2018,1,15,USD,2018-01-07,2018-04-21
2,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KOP-0014,COCO COLLECTION OIL PULLING",3/21/2018,308.56,0.154973,13,...,3,KOP-0014,COCO COLLECTION OIL PULLING,aa00f60f-d5ff-49de-8bd6-88a0434a7aca,2018,1,15,USD,2018-01-07,2018-04-21
3,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KSS-0100,SLIP INTO SUMMER KIT",3/30/2018,177.52,0.0891585,7,...,NaN,KSS-0100,SLIP INTO SUMMER KIT,aa00f60f-d5ff-49de-8bd6-88a0434a7aca,2018,1,15,USD,2018-01-07,2018-04-21
4,FULL LINE,NORTHWEST FLS,WASHINGTON/ALASKA FU,1,DT SEATTLE,"KCM-0075,COCONUT MINI MELT 2.5OZ",NaN,18,0.00904041,1,...,NaN,KCM-0075,COCONUT MINI MELT 2.5OZ,aa00f60f-d5ff-49de-8bd6-88a0434a7aca,2018,1,15,USD,2018-01-07,2018-04-21


In [167]:
def loadlocalfiletogooglestorage(batfile, source_file_name, dest_file_name):
    pass_arg=[]
    pass_arg.append(batfile)
    pass_arg.append(source_file_name)
    pass_arg.append(dest_file_name)
    p = Popen(pass_arg, stdout=PIPE, stderr=PIPE)
    output, errors = p.communicate()
    p.wait() # wait for process to terminate
    print(output)
    print(errors)

In [168]:
def loadfiletobigquery(file_name, dataset_id, table_name, delimitertype, loadtype, skipheader):
    client = bigquery.Client()
    table_ref = client.dataset(dataset_id).table(table_name)
    job_config = bigquery.LoadJobConfig()
    if skipheader is not None:
        job_config.skip_leading_rows = skipheader
    job_config.source_format = delimitertype
    #if delimitertype == bigquery.SourceFormat.CSV:
        #job_config.autodetect = True
    job_config.write_disposition = loadtype

    load_job = client.load_table_from_uri(
        file_name,
        table_ref,
        job_config=job_config)  # API request

    assert load_job.job_type == 'load'

    load_job.result()  # Waits for table load to complete.

    assert load_job.state == 'DONE'

In [169]:
print(start_date.strftime('%Y%m%d'))
print(end_date.strftime('%Y%m%d'))
print(localfilename)

20180107
20180421
C:\Users\kabhi\Desktop\datamaster\kopari\nordstrom\nordstrom_sales_20180107_20180421.csv20180107_20180421.csv


In [176]:
localfilename = app_path + filesep + 'kopari' + filesep + 'nordstrom' + filesep + 'nordstrom_sales_'
gsfilepath = gspath + gssep + 'kopari' + gssep + 'nordstrom'
localfilename = localfilename + start_date.strftime('%Y%m%d') + '_' + end_date.strftime('%Y%m%d') + '.csv'
dfdata.to_csv(localfilename, index=False)

In [177]:
loadlocalfiletogooglestorage(batfile, localfilename, gsfilepath)

b'\r\n(datamaster) C:\\Users\\kabhi\\Desktop\\datamaster>gsutil cp C:\\Users\\kabhi\\Desktop\\datamaster\\kopari\\nordstrom\\nordstrom_sales_20180107_20180421.csv gs://sarasmaster/kopari/nordstrom \r\n'
b'Copying file://C:\\Users\\kabhi\\Desktop\\datamaster\\kopari\\nordstrom\\nordstrom_sales_20180107_20180421.csv [Content-Type=application/vnd.ms-excel]...\r\n/ [0 files][    0.0 B/152.8 KiB]                                                \r/ [1 files][152.8 KiB/152.8 KiB]                                                \r\r\nOperation completed over 1 objects/152.8 KiB.                                    \r\n'


In [178]:
gsfilename = gsfilepath + gssep + 'nordstrom_sales_' + start_date.strftime('%Y%m%d') + '_' + end_date.strftime('%Y%m%d') + '.csv'
loadfiletobigquery(gsfilename, 'kopari', 'nordstrom_sales', delimitertype, loadtype, skipheader)

BadRequest: 400 Error while reading data, error message: CSV table encountered too many errors, giving up. Rows: 1; errors: 1. Please look into the error stream for more details.

In [99]:
def getlastupdateddate(dataset_name, table_name):
    client = bigquery.Client()
    query = "select max(to_date) max_updated_dt,max(to_date) min_updated_dt from " + dataset_name + "." + table_name + ";"
    job_config = bigquery.QueryJobConfig()
    query_job = client.query(query, job_config=job_config)

    query_job.result()  # Wait for job to complete

    # Print the results.
    destination_table_ref = query_job.destination
    table = client.get_table(destination_table_ref)
    table_data = None
    for row in client.list_rows(table):
        table_data = row
    return table_data

In [106]:
def checkexceptions(dataset_name, table_name, column_name):
    lastshopdate = getlastupdateddate('kopari', 'locad_tracking').max_updated_dt
    noexceptions = True if dfdata[(dfdata[column_name] > lastshopdate)].shape[0] > 0 else False
    return noexceptions